<a href="https://colab.research.google.com/github/emilyye26/Project-SPICES/blob/main/SPICES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# MIDUS NSDE Dataset
file_path = '/content/drive/My Drive/03725-0001-Data.tsv'
df = pd.read_csv(file_path, delimiter='\t')


<ipython-input-19-ab1b94667be8>:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='\t')


In [ ]:
df.head()

,M2ID,M2FAMNUM,SAMPLMAJ,A1PAGE_M2,B1PGENDER,A2DIDDAY,A2DDAY,A2DIYEAR,A2DIMON,A2DAMPM,...,A2DSYMAV,A2DSEVAV,A2DXFCAV,A2DXFDAV,A2DXFEAV,A2DXFFAV,A2DXFGAV,A2DXFHAV,A2DXFIAV,A2DXFJAV
0,10013,100004,1,27,1,100131,1,1996,8,1,...,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
1,10013,100004,1,27,1,100132,2,1996,8,1,...,0.2,2.0,2.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,10013,100004,1,27,1,100134,4,1996,8,1,...,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
3,10013,100004,1,27,1,100137,7,1996,8,1,...,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
4,10013,100004,1,27,1,100138,8,1996,8,1,...,0.4,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0


In [ ]:
df.shape

(10488, 333)

In [ ]:
list[df.select_dtypes(include=['float','int']).columns]

list[Index(['M2ID', 'M2FAMNUM', 'SAMPLMAJ', 'A1PAGE_M2', 'B1PGENDER', 'A2DIDDAY',
       'A2DDAY', 'A2DAMPM', 'A2DSTAT', 'A2DWEEKD',
       ...
       'A2DSYMAV', 'A2DSEVAV', 'A2DXFCAV', 'A2DXFDAV', 'A2DXFEAV', 'A2DXFFAV',
       'A2DXFGAV', 'A2DXFHAV', 'A2DXFIAV', 'A2DXFJAV'],
      dtype='object', length=331)]

In [ ]:
list[df.select_dtypes(include=['object']).columns]

list[Index(['A2DIYEAR', 'A2DIMON'], dtype='object')]

In [ ]:
df['A2DIYEAR'].unique()

array([1996, 1997, '1996', '1997', ' '], dtype=object)

In [ ]:
nan_count = df.isnull().sum()
nan_count

,0
M2ID,0
M2FAMNUM,0
SAMPLMAJ,0
A1PAGE_M2,0
B1PGENDER,0
...,...
A2DXFFAV,0
A2DXFGAV,0
A2DXFHAV,0
A2DXFIAV,0


In [ ]:
df.select_dtypes(include=['object']).columns.tolist()


['A2DIYEAR', 'A2DIMON']

In [ ]:
obj_columns = ['A2DIYEAR', 'A2DIMON']
df[obj_columns]


,A2DIYEAR,A2DIMON
0,1996,8
1,1996,8
2,1996,8
3,1996,8
4,1996,8
...,...,...
10483,1996,12
10484,1996,12
10485,1996,12
10486,1996,12


In [ ]:
df['A2DIYEAR'].unique()

array([1996, 1997, '1996', '1997', ' '], dtype=object)

In [ ]:
df.dropna(inplace=True)


In [ ]:
df['A2DIYEAR'].unique()

array([1996, 1997, '1996', '1997', ' '], dtype=object)

Isolating the features we selected and are going to use for our project

In [ ]:
selected_features = [
    'B1PGENDER', 'A1PAGE_M2', 'A2DD3', 'A2DA12', 'A2DA4A',
    'A2DI2A', 'A2DA2', 'A2DA3', 'A2DA13C', 'A2DF2J',
    'A2DA1', 'A2DNOALC', 'A2DNOSMO', 'A2DF2', 'A2DC1',
    'A2DI2A', 'A2DF5', 'A2DF3', 'A2DF4', 'A2DI3A',
    'A2DF3B', 'A2DTOTST'
]

Placing these features in their own dataframe

In [ ]:
df_selected = df[selected_features]

In [ ]:
print(df_selected.head())
df_selected.shape

   B1PGENDER  A1PAGE_M2  A2DD3  A2DA12  A2DA4A  A2DI2A  A2DA2  A2DA3  A2DA13C  \
0          1         27      2       2     1.0       9    0.5    0.0     99.0   
1          1         27      2       2     0.0       9    1.0    0.0     99.0   
2          1         27      2       2     0.0       9    1.0    0.0     99.0   
3          1         27      2       2     0.5       9    1.0    0.0     99.0   
4          1         27      2       2     0.0       9    0.5    9.0     99.0   

   A2DF2J  ...  A2DNOSMO  A2DF2  A2DC1  A2DI2A  A2DF5  A2DF3  A2DF4  A2DI3A  \
0       9  ...         8      2      5       9      2      2      2       9   
1       9  ...         2      1      5       9      2      2      2       9   
2       9  ...         2      2      5       9      2      2      2       9   
3       9  ...         2      2      5       9      2      2      2       9   
4       9  ...         2      2      5       9      2      2      2       9   

   A2DF3B  A2DTOTST  
0       9       

(10488, 22)

In [ ]:
df_selected.rename(columns={
    'B1PGENDER': 'Gender',
    'A1PAGE_M2': 'Age',
    'A2DD3': 'Less_Time_With_People',
    'A2DA12': 'Emotional_Support_Given',
    'A2DA4A': 'Vigorous_Exercise_Time',
    'A2DI2A': 'Health_Problems_7_Days',
    'A2DA4': 'Time_Spent_Relaxing',
    'A2DA2': 'Time_Chores',
    'A2DA3': 'Time_Work_School',
    'A2DA12D': 'Time_Give_Emotional_Support',
    'A2DA13C': 'Time_Receive_Emotional_Support',
    'A2DF2J': 'Risk_Disagreement_Plans',
    'A2DA1': 'Time_With_Children',
    'A2DNOALC': 'Never_Alcohol',
    'A2DNOSMO': 'Never_Smokes',
    'A2DF2': 'Avoided_Disagreement',
    'A2DC1': 'Depressed',
    'A2DF5': 'Discrimination',
    'A2DA5': 'Time_Sleeping',
    'A2DF3': 'Work_School_Stressors',
    'A2DF4': 'Home_Stressors',
    'A2DI3A': 'Stress_Level_7_Days',
    'A2DF3B': 'Stress_Intensity',
    'A2DTOTST': 'Summed_Stressors_Per_Day'
}, inplace=True)

print(df_selected.head())


   Gender  Age  Less_Time_With_People  Emotional_Support_Given  \
0       1   27                      2                        2   
1       1   27                      2                        2   
2       1   27                      2                        2   
3       1   27                      2                        2   
4       1   27                      2                        2   

   Vigorous_Exercise_Time  Health_Problems_7_Days  Time_Chores  \
0                     1.0                       9          0.5   
1                     0.0                       9          1.0   
2                     0.0                       9          1.0   
3                     0.5                       9          1.0   
4                     0.0                       9          0.5   

   Time_Work_School  Time_Receive_Emotional_Support  Risk_Disagreement_Plans  \
0               0.0                            99.0                        9   
1               0.0                           

<ipython-input-39-73b598fd2e7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns={


Checking for missing data in new dataframe

In [ ]:
nan_count = np.sum(df_selected.isnull(), axis = 0)

nan_count_filtered = nan_count[nan_count > 0]
nan_count_filtered

,0


In [ ]:
print(df_selected.dtypes)

Gender                              int64
Age                                 int64
Less_Time_With_People               int64
Emotional_Support_Given             int64
Vigorous_Exercise_Time            float64
Health_Problems_7_Days              int64
Time_Chores                       float64
Time_Work_School                  float64
Time_Receive_Emotional_Support    float64
Risk_Disagreement_Plans             int64
Time_With_Children                float64
Never_Alcohol                       int64
Never_Smokes                        int64
Avoided_Disagreement                int64
Depressed                           int64
Health_Problems_7_Days              int64
Discrimination                      int64
Work_School_Stressors               int64
Home_Stressors                      int64
Stress_Level_7_Days                 int64
Stress_Intensity                    int64
Summed_Stressors_Per_Day            int64
dtype: object


In [ ]:
unique_values = df_selected.nunique()
print(unique_values)

Gender                               2
Age                                 53
Less_Time_With_People                3
Emotional_Support_Given              4
Vigorous_Exercise_Time              56
Health_Problems_7_Days               4
Time_Spent_Work_School              61
Time_Receiving_Emotional_Support    92
Risk_Disagreement_Plans             50
Time_With_Children                   7
Alcohol_Never                       81
No_Alcohol                           3
No_Smoking                           3
Avoid_Disagreement                   3
Feel_Depressed                       6
Health_Problems_7_Days               4
Experienced_Discrimination           3
Stressors_Work_School                3
Stressors_Home                       3
Stress_7_Days                        4
Stress_Intensity                     5
Total_Stressors                      9
dtype: int64


Detecting outliers by calculating the z-score. We set the threshold for the z-score to be 3 (standard), and we looked for rows with outliers according to this standard. We found that every row has an outlier that we will have to handle.

In [ ]:
df_numeric = df_selected.select_dtypes(include=['number'])
df_zscores = df_numeric.apply(stats.zscore)

# set threshold for outliers (z-score > 3 or z-score < -3)
threshold = 3
outliers = df_zscores[(df_zscores > 3) | (df_zscores < -3)]

# print rows that have outliers
outlier_rows = df_numeric[(df_zscores > 3) | (df_zscores < -3)].dropna()
print(outlier_rows)
df_selected.shape

Empty DataFrame
Columns: [Gender, Age, Less_Time_With_People, Emotional_Support_Given, Vigorous_Exercise_Time, Health_Problems_7_Days, Time_Spent_Work_School, Time_Receiving_Emotional_Support, Risk_Disagreement_Plans, Time_With_Children, Alcohol_Never, No_Alcohol, No_Smoking, Avoid_Disagreement, Feel_Depressed, Health_Problems_7_Days, Experienced_Discrimination, Stressors_Work_School, Stressors_Home, Stress_7_Days, Stress_Intensity, Total_Stressors]
Index: []

[0 rows x 22 columns]


(10488, 22)

Trying to drop the rows with outliers

In [ ]:
df_no_outliers = df_selected[(np.abs(df_zscores) <= 3).all(axis=1)]
df_no_outliers.shape

(8149, 22)